<a href="https://colab.research.google.com/github/kaveriC/SCCM-Team2/blob/main/FirstEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import auth
%pip install tableone


In [14]:
# Install dependencies
# !pip install --upgrade google-cloud-bigquery pandas tableone matplotlib

# Import necessary libraries
import matplotlib.pyplot as plt
import pandas as pd
from tableone import TableOne
from google.colab import auth, files

# Authenticate to access BigQuery
auth.authenticate_user()
print('✅ Authenticated')

# Load interactive data table extension
%reload_ext google.colab.data_table


✅ Authenticated


In [15]:
from google.colab import files
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [17]:
# change this to change the right project
%env GOOGLE_CLOUD_PROJECT = sccm
from google.cloud import bigquery
client = bigquery.Client(project='sccm-465717')

env: GOOGLE_CLOUD_PROJECT=sccm


In [23]:
#!pip install google-cloud-bigquery pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

print("=== ARDS COHORT WITH PEEP CRITERIA (BigQuery) ===")
print("NMB = Neuromuscular Blockade (paralytic drugs)")
print("Adding PEEP ≥ 5 cmH2O criteria to ARDS definition...")

# Initialize BigQuery client with explicit project
project_id = 'sccm-465717'
dataset_id = 'sccm-discovery.eicu_crd_ii_v0_2_0'

# Initialize client with explicit project ID
client = bigquery.Client(project=project_id)

print(f"📊 Connected to BigQuery")
print(f"Project: {project_id}")
print(f"Dataset: {dataset_id}")

# Test connection
try:
    # Simple test query to verify connection
    test_query = f"SELECT COUNT(*) as patient_count FROM `{dataset_id}.patient` LIMIT 1"
    test_result = client.query(test_query).to_dataframe()
    print(f"✅ Connection verified: {test_result['patient_count'].iloc[0]:,} patients in database")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\n🔧 Troubleshooting steps:")
    print("1. Ensure you're authenticated: run `!gcloud auth application-default login`")
    print("2. Verify project ID and dataset name are correct")
    print("3. Check that you have BigQuery access permissions")
    exit()

# Helper function to run queries
def run_query(query, description=""):
    """Execute BigQuery and return results as DataFrame"""
    if description:
        print(f"\n🔍 {description}")

    try:
        df = client.query(query).to_dataframe()
        print(f"✅ Query completed: {len(df):,} rows returned")
        return df
    except Exception as e:
        print(f"❌ Query failed: {e}")
        return pd.DataFrame()

# Step 1: Get adult patients (≥18 years) excluding pregnancy and heart failure
print("\n--- STEP 1: ADULT PATIENTS WITH EXCLUSIONS ---")

patient_query = f"""
WITH adult_patients AS (
  SELECT
    patientunitstayid,
    patienthealthsystemstayid,
    hospitalid,
    age,
    CASE
      WHEN age = '> 89' THEN 90  -- Assign 90 to patients > 89
      WHEN SAFE_CAST(age AS INT64) IS NOT NULL THEN SAFE_CAST(age AS INT64)
      ELSE NULL
    END as age_numeric,
    gender,
    unitdischargestatus,
    unitdischargeoffset,
    hospitaldischargestatus
  FROM `{dataset_id}.patient`
  WHERE (
    age = '> 89' OR
    (SAFE_CAST(age AS INT64) IS NOT NULL AND SAFE_CAST(age AS INT64) >= 18)
  )
),

-- Identify exclusions
pregnant_patients AS (
  SELECT DISTINCT patientunitstayid
  FROM `{dataset_id}.diagnosis`
  WHERE (
    REGEXP_CONTAINS(icd9code, r'V22|V23|V24|63[0-9]|64[0-9]|65[0-9]') OR
    REGEXP_CONTAINS(LOWER(diagnosisstring), r'pregnan|gravid|maternity|obstetric|delivery|labor|gestation')
  )
),

heart_failure_patients AS (
  SELECT DISTINCT patientunitstayid
  FROM `{dataset_id}.diagnosis`
  WHERE (
    REGEXP_CONTAINS(icd9code, r'398\.91|402\.01|402\.11|402\.91|404\.01|404\.03|404\.11|404\.13|404\.91|404\.93|428') OR
    REGEXP_CONTAINS(LOWER(diagnosisstring), r'heart failure|cardiac failure|congestive heart|chf|systolic.*failure|diastolic.*failure')
  )
)

SELECT
  ap.*,
  CASE WHEN pp.patientunitstayid IS NOT NULL THEN 1 ELSE 0 END as is_pregnant,
  CASE WHEN hf.patientunitstayid IS NOT NULL THEN 1 ELSE 0 END as has_heart_failure
FROM adult_patients ap
LEFT JOIN pregnant_patients pp ON ap.patientunitstayid = pp.patientunitstayid
LEFT JOIN heart_failure_patients hf ON ap.patientunitstayid = hf.patientunitstayid
WHERE pp.patientunitstayid IS NULL
  AND hf.patientunitstayid IS NULL
"""

adult_patients_df = run_query(patient_query, "Loading adult patients with exclusions")

=== ARDS COHORT WITH PEEP CRITERIA (BigQuery) ===
NMB = Neuromuscular Blockade (paralytic drugs)
Adding PEEP ≥ 5 cmH2O criteria to ARDS definition...
📊 Connected to BigQuery
Project: sccm-465717
Dataset: sccm-discovery.eicu_crd_ii_v0_2_0
✅ Connection verified: 201,727 patients in database

--- STEP 1: ADULT PATIENTS WITH EXCLUSIONS ---

🔍 Loading adult patients with exclusions
✅ Query completed: 192,610 rows returned


In [24]:
# Step 2: Identify ARDS patients by ICD/diagnosis
print("\n--- STEP 2: ARDS BY ICD/DIAGNOSIS ---")

ards_icd_query = f"""
SELECT DISTINCT
  d.patientunitstayid,
  d.icd9code,
  d.diagnosisstring
FROM `{dataset_id}.diagnosis` d
INNER JOIN ({patient_query}) ap ON d.patientunitstayid = ap.patientunitstayid
WHERE (
  d.icd9code = '518.82' OR
  REGEXP_CONTAINS(LOWER(d.diagnosisstring), r'ards|acute respiratory distress|respiratory failure')
)
"""

ards_icd_df = run_query(ards_icd_query, "Finding ARDS patients by ICD/diagnosis")


--- STEP 2: ARDS BY ICD/DIAGNOSIS ---

🔍 Finding ARDS patients by ICD/diagnosis
✅ Query completed: 28,574 rows returned


In [25]:
# Step 3: PEEP ≥ 5 cmH2O criteria (first 48 hours)
print("\n--- STEP 3: PEEP ≥ 5 cmH2O CRITERIA ---")

peep_query = f"""
WITH peep_data AS (
  SELECT
    rc.patientunitstayid,
    rc.respchartoffset,
    SAFE_CAST(rc.respchartvalue AS FLOAT64) as peep_value
  FROM `{dataset_id}.respiratorycharting` rc
  INNER JOIN ({patient_query}) ap ON rc.patientunitstayid = ap.patientunitstayid
  WHERE rc.respchartvaluelabel = 'PEEP'
    AND rc.respchartoffset >= 0
    AND rc.respchartoffset <= 2880  -- 48 hours
    AND SAFE_CAST(rc.respchartvalue AS FLOAT64) IS NOT NULL
    AND SAFE_CAST(rc.respchartvalue AS FLOAT64) BETWEEN 0 AND 30
),

peep_summary AS (
  SELECT
    patientunitstayid,
    COUNT(*) as peep_measurements,
    MAX(peep_value) as max_peep,
    AVG(peep_value) as mean_peep,
    MAX(CASE WHEN peep_value >= 5 THEN 1 ELSE 0 END) as has_peep_ge_5
  FROM peep_data
  GROUP BY patientunitstayid
)

SELECT *
FROM peep_summary
WHERE has_peep_ge_5 = 1
"""

peep_eligible_df = run_query(peep_query, "Finding patients with PEEP ≥ 5 cmH2O")



--- STEP 3: PEEP ≥ 5 cmH2O CRITERIA ---

🔍 Finding patients with PEEP ≥ 5 cmH2O
✅ Query completed: 43,639 rows returned


In [26]:
# Step 4: S/F ratio ≤ 315 criteria
print("\n--- STEP 4: S/F RATIO ≤ 315 CRITERIA ---")

sf_ratio_query = f"""
WITH fio2_data AS (
  SELECT
    rc.patientunitstayid,
    rc.respchartoffset,
    SAFE_CAST(rc.respchartvalue AS FLOAT64) as fio2_value
  FROM `{dataset_id}.respiratorycharting` rc
  INNER JOIN ({patient_query}) ap ON rc.patientunitstayid = ap.patientunitstayid
  WHERE rc.respchartvaluelabel = 'FiO2'
    AND rc.respchartoffset >= 0
    AND rc.respchartoffset <= 2880
    AND SAFE_CAST(rc.respchartvalue AS FLOAT64) BETWEEN 21 AND 100
),

spo2_data AS (
  SELECT
    rc.patientunitstayid,
    rc.respchartoffset,
    SAFE_CAST(rc.respchartvalue AS FLOAT64) as spo2_value
  FROM `{dataset_id}.respiratorycharting` rc
  INNER JOIN ({patient_query}) ap ON rc.patientunitstayid = ap.patientunitstayid
  WHERE rc.respchartvaluelabel IN ('SpO2', 'O2 Sat', 'SaO2')
    AND rc.respchartoffset >= 0
    AND rc.respchartoffset <= 2880
    AND SAFE_CAST(rc.respchartvalue AS FLOAT64) BETWEEN 70 AND 100
),

sf_ratios AS (
  SELECT
    f.patientunitstayid,
    f.respchartoffset as fio2_offset,
    s.respchartoffset as spo2_offset,
    f.fio2_value,
    s.spo2_value,
    (s.spo2_value / f.fio2_value) * 100 as sf_ratio,
    ABS(f.respchartoffset - s.respchartoffset) as time_diff
  FROM fio2_data f
  INNER JOIN spo2_data s ON f.patientunitstayid = s.patientunitstayid
  WHERE ABS(f.respchartoffset - s.respchartoffset) <= 60  -- Within 60 minutes
),

sf_ratios_ranked AS (
  SELECT *,
    ROW_NUMBER() OVER (PARTITION BY patientunitstayid, fio2_offset ORDER BY time_diff ASC) as rn
  FROM sf_ratios
),

patient_sf_summary AS (
  SELECT
    patientunitstayid,
    COUNT(*) as sf_measurements,
    MIN(sf_ratio) as min_sf_ratio,
    MAX(CASE WHEN sf_ratio <= 315 THEN 1 ELSE 0 END) as has_sf_le_315
  FROM sf_ratios_ranked
  WHERE rn = 1  -- Take closest SpO2 measurement for each FiO2
  GROUP BY patientunitstayid
)

SELECT *
FROM patient_sf_summary
WHERE has_sf_le_315 = 1
"""

sf_eligible_df = run_query(sf_ratio_query, "Finding patients with S/F ratio ≤ 315")


--- STEP 4: S/F RATIO ≤ 315 CRITERIA ---

🔍 Finding patients with S/F ratio ≤ 315
✅ Query completed: 12,515 rows returned


In [28]:
# Step 5: Create final ARDS cohort
print("\n--- STEP 5: FINAL ARDS COHORT ---")

# Combine all criteria
ards_icd_patients = set(ards_icd_df['patientunitstayid'].unique())
sf_eligible_patients = set(sf_eligible_df['patientunitstayid'].unique())
peep_eligible_patients = set(peep_eligible_df['patientunitstayid'].unique())

print("ARDS criteria overlap:")
print(f"- ICD/Diagnosis: {len(ards_icd_patients):,} patients")
print(f"- S/F ratio ≤ 315: {len(sf_eligible_patients):,} patients")
print(f"- PEEP ≥ 5 cmH2O: {len(peep_eligible_patients):,} patients")

# Patients meeting ANY ARDS criteria
ards_any_criteria = ards_icd_patients | sf_eligible_patients
print(f"- ANY ARDS criteria: {len(ards_any_criteria):,} patients")

# Patients meeting ARDS + PEEP criteria
ards_with_peep = ards_any_criteria & peep_eligible_patients
print(f"- ARDS + PEEP ≥ 5: {len(ards_with_peep):,} patients")

# Final cohort (already filtered for adult, non-pregnant, non-HF)
final_cohort_ids = ards_with_peep & set(adult_patients_df['patientunitstayid'])

print(f"\n🎯 FINAL ARDS COHORT: {len(final_cohort_ids):,} patients")
print("✅ Adult (≥18 years)")
print("✅ ARDS (ICD codes OR S/F ≤ 315)")
print("✅ PEEP ≥ 5 cmH2O")
print("✅ No pregnancy/heart failure")


--- STEP 5: FINAL ARDS COHORT ---
ARDS criteria overlap:
- ICD/Diagnosis: 20,222 patients
- S/F ratio ≤ 315: 12,515 patients
- PEEP ≥ 5 cmH2O: 43,639 patients
- ANY ARDS criteria: 28,367 patients
- ARDS + PEEP ≥ 5: 16,622 patients

🎯 FINAL ARDS COHORT: 16,622 patients
✅ Adult (≥18 years)
✅ ARDS (ICD codes OR S/F ≤ 315)
✅ PEEP ≥ 5 cmH2O
✅ No pregnancy/heart failure


In [30]:
 # Step 6: Analyze exposures (NMB and Proning)
print("\n--- STEP 6: EXPOSURE ANALYSIS ---")

# Convert final cohort to string for SQL IN clause
final_cohort_str = ','.join([str(x) for x in final_cohort_ids])

if final_cohort_str:  # Only run if we have patients

    # NMB analysis
    nmb_query = f"""
    WITH nmb_medications AS (
      SELECT DISTINCT patientunitstayid, 'medication' as source, drugname
      FROM `{dataset_id}.medication`
      WHERE patientunitstayid IN ({final_cohort_str})
        AND REGEXP_CONTAINS(LOWER(drugname), r'cisatracurium|vecuronium|rocuronium|atracurium|succinylcholine|pancuronium')
    ),

    nmb_infusions AS (
      SELECT DISTINCT patientunitstayid, 'infusion' as source, drugname
      FROM `{dataset_id}.infusiondrug`
      WHERE patientunitstayid IN ({final_cohort_str})
        AND REGEXP_CONTAINS(LOWER(drugname), r'cisatracurium|vecuronium|rocuronium|atracurium|succinylcholine|pancuronium')
    )

    SELECT patientunitstayid, source, drugname
    FROM nmb_medications
    UNION ALL
    SELECT patientunitstayid, source, drugname
    FROM nmb_infusions
    """

    nmb_df = run_query(nmb_query, "Finding NMB exposures")
    nmb_patients = set(nmb_df['patientunitstayid'].unique()) if len(nmb_df) > 0 else set()

    # Proning analysis
    proning_query = f"""
    SELECT DISTINCT
      patientunitstayid,
      cplitemvalue
    FROM `{dataset_id}.careplangeneral`
    WHERE patientunitstayid IN ({final_cohort_str})
      AND REGEXP_CONTAINS(LOWER(cplitemvalue), r'prone|proning|position|turn|reposit')
    """

    proning_df = run_query(proning_query, "Finding proning exposures")
    proning_patients = set(proning_df['patientunitstayid'].unique()) if len(proning_df) > 0 else set()

else:
    nmb_patients = set()
    proning_patients = set()
    nmb_df = pd.DataFrame()
    proning_df = pd.DataFrame()

print(f"NMB patients: {len(nmb_patients):,}/{len(final_cohort_ids):,} ({len(nmb_patients)/max(len(final_cohort_ids),1)*100:.1f}%)")
print(f"Proning patients: {len(proning_patients):,}/{len(final_cohort_ids):,} ({len(proning_patients)/max(len(final_cohort_ids),1)*100:.1f}%)")



--- STEP 6: EXPOSURE ANALYSIS ---

🔍 Finding NMB exposures
✅ Query completed: 4,765 rows returned

🔍 Finding proning exposures
✅ Query completed: 796 rows returned
NMB patients: 3,790/16,622 (22.8%)
Proning patients: 796/16,622 (4.8%)


In [31]:
 # Step 7: Cohort characteristics and outcomes
print("\n--- STEP 7: COHORT CHARACTERISTICS ---")

if len(final_cohort_ids) > 0:
    final_cohort_df = adult_patients_df[adult_patients_df['patientunitstayid'].isin(final_cohort_ids)].copy()

    # Basic demographics
    mortality = (final_cohort_df['unitdischargestatus'] == 'Expired').sum()
    mean_age = final_cohort_df['age_numeric'].mean()
    median_los = final_cohort_df['unitdischargeoffset'].median() / 1440  # Convert to days

    print(f"📊 Cohort Characteristics:")
    print(f"- Total patients: {len(final_cohort_df):,}")
    print(f"- Mean age: {mean_age:.1f} years")
    print(f"- ICU mortality: {mortality:,}/{len(final_cohort_df):,} ({mortality/len(final_cohort_df)*100:.1f}%)")
    print(f"- Median LOS: {median_los:.1f} days")

    # Gender distribution
    gender_dist = final_cohort_df['gender'].value_counts()
    print(f"- Gender: {dict(gender_dist)}")

    # Show NMB drugs if any
    if len(nmb_df) > 0:
        print(f"\n💊 NMB Drugs Used:")
        nmb_drugs = nmb_df['drugname'].value_counts()
        for drug, count in nmb_drugs.head().items():
            print(f"  - {drug}: {count:,}")

    # Show proning details if any
    if len(proning_df) > 0:
        print(f"\n🔄 Proning Details:")
        proning_types = proning_df['cplitemvalue'].value_counts()
        for prone_type, count in proning_types.head().items():
            print(f"  - {prone_type}: {count:,}")

else:
    print("⚠️  No patients in final cohort")


--- STEP 7: COHORT CHARACTERISTICS ---
📊 Cohort Characteristics:
- Total patients: 16,622
- Mean age: 63.0 years
- ICU mortality: 4,054/16,622 (24.4%)
- Median LOS: 3.9 days
- Gender: {'Male': np.int64(9557), 'Female': np.int64(7061), 'Unknown': np.int64(4)}

💊 NMB Drugs Used:
  - rocuronium: 2,118
  - succinylcholine: 1,332
  - ROCURONIUM BROMIDE: 387
  - cisatracurium (mcg/kg/min): 258
  - cisatracurium: 253

🔄 Proning Details:
  - Turn/ROM: 796


In [36]:
 # Step 8: Research feasibility assessment
print("\n=== RESEARCH FEASIBILITY ASSESSMENT ===")

total_exposures = len(nmb_patients) + len(proning_patients)
overlap = len(nmb_patients & proning_patients)

print(f"📈 Exposure Summary:")
print(f"- NMB only: {len(nmb_patients - proning_patients):,}")
print(f"- Proning only: {len(proning_patients - nmb_patients):,}")
print(f"- Both exposures: {overlap:,}")
print(f"- Any exposure: {len(nmb_patients | proning_patients):,}")
print(f"- No exposures: {len(final_cohort_ids) - len(nmb_patients | proning_patients):,}")

# Feasibility recommendations
if len(final_cohort_ids) >= 100 and total_exposures >= 20:
    print("\n✅ EXCELLENT: Large cohort with sufficient exposures for robust analysis")
elif len(final_cohort_ids) >= 50 and total_exposures >= 10:
    print("\n✅ GOOD: Adequate sample size for descriptive analysis")
elif len(final_cohort_ids) >= 25 and total_exposures >= 5:
    print("\n⚠️  MODERATE: Limited but feasible for case series")
else:
    print("\n❌ LIMITED: Consider expanding inclusion criteria or focusing on descriptive analysis")

# Save results
print("\n=== SAVING RESULTS ===")

# Create summary dataframe
summary_data = {
    'metric': [
        'total_adult_patients', 'excluded_patients', 'ards_icd_patients',
        'ards_sf_patients', 'ards_peep_patients', 'final_cohort_patients',
        'nmb_patients', 'proning_patients', 'mortality_count'
    ],
    'count': [
        len(adult_patients_df),
        len(adult_patients_df) - len(final_cohort_ids),  # Rough estimate
        len(ards_icd_patients),
        len(sf_eligible_patients),
        len(peep_eligible_patients),
        len(final_cohort_ids),
        len(nmb_patients),
        len(proning_patients),
        mortality if len(final_cohort_ids) > 0 else 0
    ]
}

summary_df = pd.DataFrame(summary_data)
print("✅ Analysis complete!")
print("\nSummary Statistics:")
print(summary_df.to_string(index=False))

print("\n🔍 Next Steps:")
print("1. Export final_cohort_ids for detailed analysis")
print("2. Analyze timing of exposures relative to ARDS onset")
print("3. Perform outcome analysis with propensity matching")
print("4. Consider sensitivity analyses with different PEEP thresholds")

# Export final cohort IDs
final_cohort_list = list(final_cohort_ids)
print(f"\n📁 Final cohort patient IDs: {len(final_cohort_list):,} patients")
# Uncomment to save: pd.DataFrame({'patientunitstayid': final_cohort_list}).to_csv('ards_final_cohort.csv', index=False)


=== RESEARCH FEASIBILITY ASSESSMENT ===
📈 Exposure Summary:
- NMB only: 3,317
- Proning only: 323
- Both exposures: 473
- Any exposure: 4,113
- No exposures: 12,509

✅ EXCELLENT: Large cohort with sufficient exposures for robust analysis

=== SAVING RESULTS ===
✅ Analysis complete!

Summary Statistics:
               metric  count
 total_adult_patients 192610
    excluded_patients 175988
    ards_icd_patients  20222
     ards_sf_patients  12515
   ards_peep_patients  43639
final_cohort_patients  16622
         nmb_patients   3790
     proning_patients    796
      mortality_count   4054

🔍 Next Steps:
1. Export final_cohort_ids for detailed analysis
2. Analyze timing of exposures relative to ARDS onset
3. Perform outcome analysis with propensity matching
4. Consider sensitivity analyses with different PEEP thresholds

📁 Final cohort patient IDs: 16,622 patients
